# gcodeCutter

In [1]:
import os

In [2]:
# General information for the notebook.

# Get working directory.
cwd = os.getcwd()
print(cwd)

# Name of the file to be shortened.
file_a = "Test_file.gcode"

D:\Git\gcodeCutter


In [250]:
def cut_layers(in_file_name, 
               layer_reduction=None, 
               height_reduction=None):
    """
    ";Layer height: 0.2"
    ";LAYER_COUNT:573"
    ";LAYER:0"
    ";TIME_ELAPSED:"


    ";LAYER:1"
    "M140 S60"
    "G0 F3750 X58.618 Y27.157 Z0.4"
    """
    
    layer_start_marker = ";LAYER:"
    layer_end_marker = ";TIME_ELAPSED:"
    height_marker = "Z"
    
    file_type = in_file_name.split('.')[-1]
    file_name = in_file_name.split('.')[0]
    
    with open(in_file_name, "r") as in_file:
        in_file_content = []
        for line in in_file:
            in_file_content.append(line)
    
    
    # Get layer count to check.
    layer_count = 1
    for line in in_file_content:
        if ';LAYER:' in line:
            # Count ';LAYER:'.
            layer_count += 1
        elif ";LAYER_COUNT:" in line:
            # Read layer count from file.
            layer_count_from_file = line.split(':')[1]
            
    print('Layer count: {}'.format(layer_count))
    print('Layer count, from file: {}'.format(layer_count_from_file))
    
    
    # Sort information into head, tail and layers.
    
    # Initialise lists to collect the information.
    head_lines = list()
    layer_lines = list()
    single_layer = list()
    tail_lines = list()
    
    # Initialise flags to control sorting the content.
    read_header = True
    read_layers = False
    read_tail = False
    
    # layer counter.
    counter = 0
    
    # Iterate over all lines.
    for line in in_file_content:
        # Read header.
        if read_header is True:

            if ";LAYER_COUNT:" in line:
                # Read layer count from file.
                layer_count_from_file = line.split(':')[1]
                
            elif ";Layer height:" in line:
                # Read layer thickness.
                layer_height = float(line.split(':')[-1])
                print("Layer height from file: {} mm.".format(layer_height))
            
            elif layer_start_marker in line:
                # Switch to read layers.
                read_header = False
                read_layers = True
                single_layer.append(line)
                continue

            head_lines.append(line)
        
        # Read layers.
        elif read_layers is True:
            single_layer.append(line)
            
            if layer_end_marker in line:
                layer_lines.append(single_layer)
                single_layer = list()
                counter += 1
#                 print(counter, layer_count_from_file)
                
            if counter == int(layer_count_from_file):
                read_layers = False
                read_tail = True
                print('* {} layers have been read.'.format(counter))
                continue
                
            
        # Read tail.
        elif read_tail is True:
            
            tail_lines.append(line)
    
    
    # Remove layers.
    
    # Calculate Z value to be cut off.
    z_cut = layer_reduction * layer_height
    print("Z value to be cut off: {} mm   ({} mm * {})".format(z_cut, 
                                                            layer_height, 
                                                            layer_reduction))
    
    # Initialise list for reduced layer blocks.
    new_line_blocks = list()
    
    new_layer_count = 0
    for layer_block in layer_lines[layer_reduction:]:
        
        for line_id, line in enumerate(layer_block):
            if height_marker in line:
                # 'G0 X103.039 Y106.9 Z3\n',
                z_line = line.split(' ')
                original_z = z_line[-1][1:-1]
                print('original Z: {}'.format(original_z))
                
                new_z = round(float(original_z) - z_cut, 1)
                print('new Z:      {} \n'.format(new_z))
                
                print(z_line)
                z_line[-1] = "Z{}\n".format(new_z)
                print(z_line)
                
                # Create new layer label.
                new_layer_label = ";LAYER:{}\n".format(new_layer_count)
                print(new_layer_label)
                layer_block[0] = new_layer_label
                
                # Build new string.
                new_z_line = z_line[0]
                for element in z_line[1:]:
                    new_z_line += ' '
                    new_z_line += element
                    
                layer_block[line_id] = new_z_line
        new_line_blocks.append(layer_block)
        
        # Advance layer counter.
        new_layer_count += 1
    
    
    
    # Build output file content.
    outfile_content = list()
    
    # Add header lines.
    for line in head_lines:
        outfile_content.append(line)
    
    # Add new layer blocks.
    for block in new_line_blocks:
        for line in block:
            outfile_content.append(line)
            
    # Add tail lines.
    for line in tail_lines:
        outfile_content.append(line)
        
        
    # Write new file.
    
    # Build new file name.
    outfile_name = "{}_short.{}".format(file_name, file_type)
    outfile_path = os.path.join(cwd, outfile_name)
    
    with open(outfile_path, 'w') as out_file:
        for line in outfile_content:
            out_file.write(line)#("%s\n" % item)

    return head_lines, layer_lines, tail_lines, new_line_blocks
    
    

In [253]:

headlines, layerlines, taillines, newblocks = cut_layers(in_file_name=file_a,
                                              layer_reduction=5,
                                              height_reduction=None)

Layer count: 16
Layer count, from file: 15

Layer height from file: 0.2 mm.
* 15 layers have been read.
Z value to be cut off: 1.0 mm   (0.2 mm * 5)
original Z: 1.2
new Z:      0.2 

['G0', 'X103.439', 'Y106.5', 'Z1.2\n']
['G0', 'X103.439', 'Y106.5', 'Z0.2\n']
;LAYER:0

original Z: 1.4
new Z:      0.4 

['G0', 'X103.439', 'Y106.5', 'Z1.4\n']
['G0', 'X103.439', 'Y106.5', 'Z0.4\n']
;LAYER:1

original Z: 1.6
new Z:      0.6 

['G0', 'X103.439', 'Y106.5', 'Z1.6\n']
['G0', 'X103.439', 'Y106.5', 'Z0.6\n']
;LAYER:2

original Z: 1.8
new Z:      0.8 

['G0', 'X103.439', 'Y106.5', 'Z1.8\n']
['G0', 'X103.439', 'Y106.5', 'Z0.8\n']
;LAYER:3

original Z: 2
new Z:      1.0 

['G0', 'X103.439', 'Y106.5', 'Z2\n']
['G0', 'X103.439', 'Y106.5', 'Z1.0\n']
;LAYER:4

original Z: 2.2
new Z:      1.2 

['G0', 'X103.039', 'Y106.9', 'Z2.2\n']
['G0', 'X103.039', 'Y106.9', 'Z1.2\n']
;LAYER:5

original Z: 2.4
new Z:      1.4 

['G0', 'X103.039', 'Y106.9', 'Z2.4\n']
['G0', 'X103.039', 'Y106.9', 'Z1.4\n']
;LAYER:6

o

In [254]:
newblocks

[[';LAYER:0\n',
  ';MESH:edbacba6-527a-45ff-9e99-ffa36881277f.stl\n',
  'G0 X103.439 Y106.5 Z0.2\n',
  'M204 S1800\n',
  'M205 X8 Y8\n',
  ';TYPE:WALL-INNER\n',
  'G1 F1500 X103.439 Y103.5 E24.99219\n',
  'G1 X105.099 Y103.5 E25.0474\n',
  'G1 X105.099 Y106.5 E25.14718\n',
  'G1 X103.439 Y106.5 E25.20239\n',
  'M204 S3000\n',
  'M205 X10 Y10\n',
  'G0 F6000 X103.039 Y106.9\n',
  'M204 S1800\n',
  'M205 X8 Y8\n',
  'G1 F1500 X103.039 Y103.1 E25.32878\n',
  'G1 X105.499 Y103.1 E25.4106\n',
  'G1 X105.499 Y106.9 E25.53699\n',
  'G1 X103.039 Y106.9 E25.61881\n',
  'M204 S3000\n',
  'M205 X10 Y10\n',
  'G0 F6000 X102.639 Y107.3\n',
  'M204 S1800\n',
  'M205 X8 Y8\n',
  ';TYPE:WALL-OUTER\n',
  'G1 F1500 X102.639 Y102.7 E25.77181\n',
  'G1 X105.899 Y102.7 E25.88023\n',
  'G1 X105.899 Y107.3 E26.03323\n',
  'G1 X102.639 Y107.3 E26.14166\n',
  'M204 S3000\n',
  'M205 X10 Y10\n',
  'G0 F6000 X102.639 Y107.1\n',
  'G0 X103.109 Y106.83\n',
  'G0 X103.829 Y106.11\n',
  'M204 S1800\n',
  'M205 X8 Y8

In [73]:
headlines

[';FLAVOR:Marlin\n',
 ';TIME:61\n',
 ';Filament used: 0.0369016m\n',
 ';Layer height: 0.2\n',
 ';MINX:94.919\n',
 ';MINY:94.98\n',
 ';MINZ:0.2\n',
 ';MAXX:115.081\n',
 ';MAXY:115.02\n',
 ';MAXZ:3\n',
 ';Generated with Cura_SteamEngine 4.1.0\n',
 'M140 S60\n',
 'M105\n',
 'M190 S60\n',
 'M104 S205\n',
 'M105\n',
 'M109 S205\n',
 'M82 ;absolute extrusion mode\n',
 'G21 ;metric values\n',
 'G90 ;absolute positioning\n',
 'M82 ;set extruder to absolute mode\n',
 'M107 ;start with the fan off\n',
 'G28 X0 Y0 ;move X/Y to min endstops\n',
 'G28 Z0 ;move Z to min endstops\n',
 'G1 Z15.0 F100 ;move the platform down 15mm\n',
 'G92 E0 ;zero the extruded length\n',
 'G1 F200 E3 ;extrude 3mm of feed stock\n',
 'G92 E0 ;zero the extruded length again\n',
 'G1 F100\n',
 'M117 Printing...\n',
 'G5\n',
 'G92 E0\n',
 'G92 E0\n',
 'G1 F2400 E-6\n',
 ';LAYER_COUNT:15\n']

In [249]:
layerlines[9]

[';LAYER:5\n',
 ';MESH:edbacba6-527a-45ff-9e99-ffa36881277f.stl\n',
 'G0 X103.439 Y106.5 Z1.0\n',
 'M204 S1800\n',
 'M205 X8 Y8\n',
 ';TYPE:WALL-INNER\n',
 'G1 F1500 X103.439 Y103.5 E31.18418\n',
 'G1 X105.099 Y103.5 E31.23939\n',
 'G1 X105.099 Y106.5 E31.33917\n',
 'G1 X103.439 Y106.5 E31.39438\n',
 'M204 S3000\n',
 'M205 X10 Y10\n',
 'G0 F6000 X103.039 Y106.9\n',
 'M204 S1800\n',
 'M205 X8 Y8\n',
 'G1 F1500 X103.039 Y103.1 E31.52077\n',
 'G1 X105.499 Y103.1 E31.60259\n',
 'G1 X105.499 Y106.9 E31.72898\n',
 'G1 X103.039 Y106.9 E31.8108\n',
 'M204 S3000\n',
 'M205 X10 Y10\n',
 'G0 F6000 X102.639 Y107.3\n',
 'M204 S1800\n',
 'M205 X8 Y8\n',
 ';TYPE:WALL-OUTER\n',
 'G1 F1500 X102.639 Y102.7 E31.96379\n',
 'G1 X105.899 Y102.7 E32.07222\n',
 'G1 X105.899 Y107.3 E32.22522\n',
 'G1 X102.639 Y107.3 E32.33365\n',
 'M204 S3000\n',
 'M205 X10 Y10\n',
 'G0 F6000 X102.639 Y107.1\n',
 'G0 X103.109 Y106.83\n',
 'G0 X103.829 Y106.11\n',
 'M204 S1800\n',
 'M205 X8 Y8\n',
 ';TYPE:SKIN\n',
 'G1 F600 X10

In [75]:
taillines

['G1 F2400 E30.90163\n',
 'M140 S0\n',
 'M204 S4000\n',
 'M205 X20 Y20\n',
 'M107\n',
 'M104 S0 ; turn off extruder\n',
 'M140 S0 ; turn off bed\n',
 'M84 ; disable motors\n',
 'M107\n',
 'G91 ;relative positioning\n',
 'G1 E-1 F300 ;retract the filament a bit before lifting the nozzle, to release some of the pressure\n',
 'G1 Z+0.5 E-5 ;X-20 Y-20 F100 ;move Z up a bit and retract filament even more\n',
 'G28 X0 ;Y0 ;move X/Y to min endstops, so the head is out of the way\n',
 'G1 Y180 F2000\n',
 'M84 ;steppers off\n',
 'G90\n',
 'M300 P300 S4000\n',
 'M82 ;absolute extrusion mode\n',
 'M104 S0\n',
 ';End of Gcode\n',
 ';SETTING_3 {"extruder_quality": ["[general]\\\\nversion = 4\\\\nname = Normal #2\\\\nd\n',
 ';SETTING_3 efinition = anycubic_i3_mega\\\\n\\\\n[metadata]\\\\ntype = quality_changes\n',
 ';SETTING_3 \\\\nsetting_version = 7\\\\nposition = 0\\\\nquality_type = normal\\\\n\\\\n[v\n',
 ';SETTING_3 alues]\\\\ndefault_material_print_temperature = 205\\\\ninfill_overlap = \n',
